In [52]:
from IPython.display import IFrame
IFrame("homework1.pdf", width=600, height=300)

In [96]:
#importing libraries
import numpy as np
import json
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")


In [29]:
#utility function for parsing JSONdata
def parseData(fname):
    for l in fname:
        yield eval(l)

In [30]:
file = open('beer_50000.json')

In [31]:
# loading and parsing JSONdata
data  =list(parseData(file))

In [32]:
type(data)

list


# Tasks — Regression (week 1):

# 1.No of Reviews for each style of beer in the dataset (`beer/style`) ,  
# avg value of `review/taste` for reviews from each style  

In [33]:
def avgReview(dict):
    total = dict()
    freq = dict()
    avg = dict()
    for dict_ in data:
        if dict_['beer/style'] not in total:
            total[dict_['beer/style']] = 0
            freq[dict_['beer/style']]  = 0
            
        total[dict_['beer/style']] += dict_['review/taste']
        freq [dict_['beer/style']] += 1
    for style in total:
        avg[style] = round(total[style]/freq[style], 6)
    return avg, freq

In [34]:
avg ,freq = avgReview(dict)

In [35]:
freq

{'Altbier': 165,
 'American Adjunct Lager': 242,
 'American Amber / Red Ale': 665,
 'American Amber / Red Lager': 42,
 'American Barleywine': 825,
 'American Black Ale': 138,
 'American Blonde Ale': 357,
 'American Brown Ale': 314,
 'American Dark Wheat Ale': 14,
 'American Double / Imperial IPA': 3886,
 'American Double / Imperial Pilsner': 14,
 'American Double / Imperial Stout': 5964,
 'American IPA': 4113,
 'American Malt Liquor': 90,
 'American Pale Ale (APA)': 2288,
 'American Pale Lager': 123,
 'American Pale Wheat Ale': 154,
 'American Porter': 2230,
 'American Stout': 591,
 'American Strong Ale': 166,
 'American Wild Ale': 98,
 'Baltic Porter': 514,
 'Belgian Dark Ale': 175,
 'Belgian IPA': 128,
 'Belgian Pale Ale': 144,
 'Belgian Strong Dark Ale': 146,
 'Belgian Strong Pale Ale': 632,
 'Berliner Weissbier': 10,
 'BiÃ¨re de Garde': 7,
 'Black & Tan': 122,
 'Bock': 148,
 'Braggot': 26,
 'California Common / Steam Beer': 11,
 'Chile Beer': 11,
 'Cream Ale': 69,
 'Czech Pilsener'

In [36]:
avg

{'Altbier': 3.40303,
 'American Adjunct Lager': 2.948347,
 'American Amber / Red Ale': 3.513534,
 'American Amber / Red Lager': 3.690476,
 'American Barleywine': 4.064242,
 'American Black Ale': 3.873188,
 'American Blonde Ale': 3.254902,
 'American Brown Ale': 3.743631,
 'American Dark Wheat Ale': 3.678571,
 'American Double / Imperial IPA': 4.033325,
 'American Double / Imperial Pilsner': 3.821429,
 'American Double / Imperial Stout': 4.479963,
 'American IPA': 4.000851,
 'American Malt Liquor': 2.255556,
 'American Pale Ale (APA)': 3.649694,
 'American Pale Lager': 3.215447,
 'American Pale Wheat Ale': 3.334416,
 'American Porter': 4.081839,
 'American Stout': 3.819797,
 'American Strong Ale': 3.569277,
 'American Wild Ale': 4.188776,
 'Baltic Porter': 4.213035,
 'Belgian Dark Ale': 3.34,
 'Belgian IPA': 3.949219,
 'Belgian Pale Ale': 3.739583,
 'Belgian Strong Dark Ale': 3.695205,
 'Belgian Strong Pale Ale': 4.056171,
 'Berliner Weissbier': 3.55,
 'BiÃ¨re de Garde': 3.928571,
 'Bla

# 2. simple predictor with a single binary feature indicating whether a beer is an `American IPA`

In [37]:
X = []
y = []
for d in data:
    if d['beer/style'] == 'American IPA':
        X.append([1, 1])
    else:
        X.append([1, 0])
    y.append(d['review/taste'])

In [38]:
X = np.array(X)
y = np.array(y)


In [39]:
theta, reiduals, rank, s = np.linalg.lstsq(X, y)

In [40]:
theta[0], theta[1]


(3.9152047420838927, 0.08564621828574305)

theta[0] represent the y-intercept of the regression line

theta[1] represent the slope of the regression line

# 3.  Splitting the data into train-test sets , Finding MSE on training and testing set

In [41]:
X.shape

(50000, 2)

In [42]:
y.shape

(50000,)

In [43]:
X_train, X_test = X[:int(len(X)/2),:], X[int(len(X)/2):,:]

In [44]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]

In [45]:
# linear Regression
reg = LinearRegression()
reg.fit(X_train, y_train)
print("coeficients: ",reg.coef_)
print("intercept: ",reg.intercept_)
print("MSE on train set: ",MSE(y_train, reg.predict(X_train)) )
print("MSE on test set: ",MSE(y_test, reg.predict(X_test)))
print("r2_score on train set:",r2_score(y_train, reg.predict(X_train)) )
print("r2_score on test set:",r2_score(y_test, reg.predict(X_test)) )

coeficients:  [0.         0.05606027]
intercept:  3.90435639229422
MSE on train set:  0.558107286558669
MSE on test set:  0.4684100509666458
r2_score on train set: 0.0004442931134949202
r2_score on test set: 7.22798026219218e-05


# 4. Extending the above model , incorporating binary features for every style of beer with ≥ 50 reviews

In [46]:
beer_rev50 = []
for beer_style in freq.keys():
    if freq[beer_style] > 50:
        beer_rev50.append(beer_style)
        
print( "beer styles with reviews count > 50 are :",len(beer_rev50))
X = []
y = []
for d in data:
    x = []
    for beer in beer_rev50:
        x.append( d['beer/style'] == beer )
    X.append(x)
    y.append(d['review/taste'])


beer styles with reviews count > 50 are : 74


In [47]:
X = np.array(X)
y = np.array(y)

In [48]:
X.shape, y.shape

((50000, 74), (50000,))

In [49]:
X_train, X_test = X[:int(len(X)/2),:], X[int(len(X)/2):,:]

In [50]:
y_train, y_test = y[:int(len(X)/2)], y[int(len(X)/2):]

In [51]:
# linear Regression
reg = LinearRegression()
reg.fit(X_train, y_train)
print("coeficients: ",reg.coef_)
print("intercept: ",reg.intercept_)
print("MSE on training set: ",MSE(y_train, reg.predict(X_train)) )
print("MSE on testing set: ",MSE(y_test, reg.predict(X_test)) )
print("r2_score on train set:",r2_score(y_train, reg.predict(X_train)) )
print("r2_score on test set:",r2_score(y_test, reg.predict(X_test)) )

coeficients:  [ 0.76540404  0.45638407 -0.74318182  0.35359848 -0.45410882  0.12193999
  0.69564175  0.12395105  0.12651515 -0.50681818 -1.00681818 -0.10223103
 -0.1798951  -0.58598485 -0.0058248  -0.4664673   0.33580477  0.3449362
 -0.27061129 -0.35681818  0.65508658  0.44318182  0.71136364  0.20984848
 -0.09003966  0.19621212 -0.73802386  0.01699134  0.20312334  0.13786267
  0.02739474 -0.31931818  0.26709486 -0.38622995  0.3763751   0.26818182
 -0.17824675 -0.22045455 -0.20681818  0.46842454 -0.92019846  0.60472028
  0.09815076  0.26761104  0.39318182  0.25681818 -0.10681818  0.1527972
  0.01818182 -0.65227273 -0.11634199  0.3305986  -0.13106061 -0.18884943
 -0.63806818 -1.23390152  0.33596789 -0.41285266  0.11320382  0.22651515
  0.05895722  0.8416167  -0.83277972  0.38356643  0.58195733 -0.02450111
  0.10049889 -0.2937747   0.48544372  0.03420746  0.14466111  0.11433566
  0.16385851 -0.62765152]
intercept:  3.60681818181818
MSE on training set:  0.3678402770901444
MSE on testing s

# Tasks — Classification (week 2):

# 5. Training a predictor that estimates whether a beer is an `American IPA` using two features:
[`beer/ABV`,`review/taste`]

In [89]:
def generateFeatures(data):
    X = []
    y = []
    for d in data:
        X.append([d['beer/ABV'], d['review/taste']])
        y.append(d['beer/style'] == 'American IPA')
    return X, y

In [90]:
X , y = generateFeatures(data)

In [91]:
len(X), len(y)

(50000, 50000)

In [92]:
X_train, X_test = X[:int(len(X)/2)], X[int(len(X)/2):]

In [93]:
y_train, y_test = y[:int(len(y)/2)], y[int(len(y)/2):]

In [95]:
clf = svm.SVC(C=1000)
clf.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [103]:
train_acc = accuracy_score(y_train, clf.predict(X_train))
print("Train set accuracy score", train_acc )

Train set accuracy score 0.9226


In [104]:
test_acc = accuracy_score(y_test, clf.predict(X_test))
print("Train set accuracy score", test_acc )

Train set accuracy score 0.85632
